# Stochastic Network Optimization with PyPSA

This notebook demonstrates principles of stochastic optimization for energy system planning under uncertainty and their implementation in PyPSA.

## Overview

Stochastic optimization in PyPSA enables modeling and solving power system planning problems under uncertainty. This capability addresses real-world scenarios where parameters such as fuel prices, renewable energy availability, demand patterns, or technology costs are uncertain at the time investment decisions must be made.

PyPSA implements a **two-stage stochastic programming framework** with scenario trees, allowing users to optimize investment decisions (first-stage) that are robust across multiple possible future realizations (scenarios) of uncertain parameters.

### What This Notebook Contains

- Mathematical formulation of two-stage stochastic programming
- Define a toy optimization problem to illustrate the concept
- Part 1: Deterministic optimization over individual scenarios
- Part 2: Stochastic optimization using PyPSA API
- Part 3: Manual stochastic model construction with linopy
- Value of information analysis (EVPI, ECIU)
- Economic interpretation and references to good literature
- Technical details and coming features in future PyPSA releases

## Mathematical Foundation

### Two-Stage Stochastic Programming

The stochastic optimization problem follows the standard formulation:

$$
\begin{align}
\min_{x} \quad & c^T x + \sum_{s \in S} p_s Q(x, \xi_s) \\
\text{s.t.} \quad & A x = b \\
& x \geq 0
\end{align}
$$

Where:
- $x$: First-stage decisions (investment in generation/transmission/storage)
- $\xi_s$: Random parameter realizations in scenario $s$
- $p_s$: Probability of scenario $s$
- $Q(x, \xi_s)$: Second-stage recourse function
- $S$: Set of all scenarios

The second-stage problem represents:

$$
\begin{align}
Q(x, \xi_s) = \min_{y_s} \quad & q_s^T y_s \\
\text{s.t.} \quad & W y_s = h_s - T x \\
& y_s \geq 0
\end{align}
$$

Where:

- $y_s$: Second-stage (wait-and-see) decision variables for scenario $s$ (e.g., energy dispatch decisions)
- $q_s$: Second-stage cost coefficients for scenario $s$
- $W$: Recourse matrix (coefficient matrix for second-stage variables $y_s$)
- $T$: Technology matrix (coefficient matrix for first-stage variables $x$ in second-stage constraints)
- $h_s$: Right-hand-side vector for scenario $s$

The second-stage constraints $W y_s = h_s - T x$ show how first-stage decisions $x$ affect the feasible region for second-stage variables $y_s$ in each scenario.

For a comprehensive guide into the stochastic programming theory and methods, see Birge and Louveaux (2011) [1].

## Imports

**Note:** This notebook requires PyPSA with stochastic optimization support. 
- For local use: `pip install git+https://github.com/PyPSA/PyPSA.git@new-opt-stoch`
- The online documentation automatically uses the correct branch

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from linopy.expressions import merge
from xarray import DataArray

import pypsa
from pypsa.components.common import as_components

## Define Toy Problem

**Note:** This toy problem setup matches the `stochastic_benchmark_network` fixture used in PyPSA's test suite (`test/conftest.py`) for validating stochastic optimization functionality

In [ ]:
# Scenario definitions - Gas price uncertainty
SCENARIOS = ["low", "med", "high"]
GAS_PRICES = {"low": 40, "med": 70, "high": 100}  # EUR/MWh_th
PROB = {"low": 0.4, "med": 0.3, "high": 0.3}  # Scenario probabilities
BASE = "low"  # Base scenario for network construction

# System parameters
FREQ = "3h"  # Time resolution
LOAD_MW = 1  # Constant load (MW)
SOLVER = "highs"  # Optimization solver

# Time series data URL
TS_URL = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)

# Load and process time series data
ts = pd.read_csv(TS_URL, index_col=0, parse_dates=True)
ts = ts.resample(FREQ).asfreq()  # Resample to 3-hour resolution


# Technology specifications
def annuity(life, rate):
    """Compute capital recovery factor for annualizing investment costs."""
    return rate / (1 - (1 + rate) ** -life) if rate else 1 / life


# Technology data: investment costs, efficiencies, marginal costs
TECH = {
    "solar": {"profile": "solar", "inv": 1e6, "m_cost": 0.01},
    "wind": {"profile": "onwind", "inv": 2e6, "m_cost": 0.02},
    "gas": {"inv": 7e5, "eff": 0.6},
    "lignite": {"inv": 1.3e6, "eff": 0.4, "m_cost": 130},
}

# Financial parameters
FOM, DR, LIFE = 3.0, 0.03, 25  # Fixed O&M (%), discount rate, lifetime (years)

# Calculate annualized capital costs
for cfg in TECH.values():
    cfg["fixed_cost"] = (annuity(LIFE, DR) + FOM / 100) * cfg["inv"]

## Helper Functions for Visualization

In [ ]:
# Color mapping for visualizations
COLOR_MAP = {
    "solar": "gold",
    "wind": "skyblue",
    "gas": "brown",
    "lignite": "black",
}


def plot_capacity(
    df,
    title="Capacity Mix",
    xlabel="Scenario",
    ylabel="Capacity (MW)",
    figsize=(8, 4),
    color_map=None,
    rotation=0,
):
    """Plot capacity mix as stacked bar chart"""
    if color_map is None:
        color_map = COLOR_MAP

    colors = [color_map.get(c, "gray") for c in df.columns]
    ax = df.plot(kind="bar", stacked=True, figsize=figsize, color=colors)
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.xticks(rotation=rotation, ha="right" if rotation > 0 else "center")
    plt.tight_layout()
    return ax


def plot_cost(
    series,
    title="Total Cost",
    xlabel="Scenario",
    ylabel="Cost (EUR/year)",
    figsize=(6, 4),
    rotation=0,
):
    """Plot costs as bar chart"""
    ax = series.plot(kind="bar", figsize=figsize, color="steelblue")
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    # Format y-axis as millions
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x / 1e6:.1f}M"))
    plt.xticks(rotation=rotation, ha="right" if rotation > 0 else "center")
    plt.tight_layout()
    return ax

## Create PyPSA network for the toy problem

In [ ]:
def build_network(gas_price):
    """
    Create PyPSA network with:
    - Single bus (DE) with constant load
    - Extendable generators: solar, wind, gas, lignite
    """
    n = pypsa.Network()
    n.set_snapshots(ts.index)
    n.snapshot_weightings = pd.Series(int(FREQ[:-1]), index=ts.index)  # 3-hour weights

    # Add bus and load
    n.add("Bus", "DE")
    n.add("Load", "DE_load", bus="DE", p_set=LOAD_MW)

    # Add renewable generators (variable renewable energy)
    for tech in ["solar", "wind"]:
        cfg = TECH[tech]
        n.add(
            "Generator",
            tech,
            bus="DE",
            p_nom_extendable=True,
            p_max_pu=ts[cfg["profile"]],  # Renewable availability profile
            capital_cost=cfg["fixed_cost"],
            marginal_cost=cfg["m_cost"],
        )

    # Add conventional generators (dispatchable)
    for tech in ["gas", "lignite"]:
        cfg = TECH[tech]
        # Gas marginal cost depends on gas price and efficiency
        mc = (gas_price / cfg.get("eff")) if tech == "gas" else cfg["m_cost"]
        n.add(
            "Generator",
            tech,
            bus="DE",
            p_nom_extendable=True,
            efficiency=cfg.get("eff"),
            capital_cost=cfg["fixed_cost"],
            marginal_cost=mc,
        )
    return n

## Part 1: Deterministic Optimization

First, we solve separate deterministic optimization problems for each gas price scenario. This represents the approach where we assume perfect foresight on the uncertain parameter. Put simply, we solve the problem for each scenario independently.

In [ ]:
# Solve deterministic problems for each scenario
caps_det = pd.DataFrame(index=SCENARIOS, columns=TECH.keys())
objs_det = pd.Series(index=SCENARIOS)

print("Solving deterministic optimization for each scenario...")
for sc in SCENARIOS:
    print(f"  Scenario '{sc}': Gas price = {GAS_PRICES[sc]} EUR/MWh_th")
    n = build_network(GAS_PRICES[sc])
    n.optimize(solver_name=SOLVER, solver_options={"log_to_console": False})
    caps_det.loc[sc] = n.generators.p_nom_opt
    objs_det.loc[sc] = n.objective

In [ ]:
# Visualize deterministic results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Capacity mix
colors = [COLOR_MAP.get(c, "gray") for c in caps_det.columns]
caps_det.plot(kind="bar", stacked=True, ax=ax1, color=colors)
ax1.set_title("Deterministic Capacity Mix by Scenario")
ax1.set_ylabel("Capacity (MW)")
ax1.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

# Total costs
(objs_det / 1e6).plot(kind="bar", ax=ax2, color="steelblue")
ax2.set_title("Deterministic Total Costs by Scenario")
ax2.set_ylabel("Total Cost (M€/year)")
ax2.tick_params(axis="x", rotation=0)

plt.tight_layout()
plt.show()

### Analysis of Deterministic Results

- Higher gas prices lead to more renewable capacity and less gas capacity
- Total system costs increase with gas prices
- Each scenario represents optimal capacity mix under perfect foresight

In [ ]:
# Inspect the deterministic problem structure
n.model

## Part 2: Stochastic Optimization with PyPSA API

The key to stochastic optimization in PyPSA is the `set_scenarios()` method. This method transforms a regular PyPSA network into a stochastic network by:

1. **Adding scenario dimensions** to all component static and time-series data
2. **Enabling scenario-specific parameters** that can vary between different uncertainty realizations
4. Facilitates **automatic formulation** of the two-stage stochastic programming problem during optimization

Once scenarios are set, PyPSA treats investment decisions (like generator/storage/line capacity) as **first-stage variables** that must be the same across all scenarios*, while operational decisions (like dispatch) become **second-stage variables** that can differ by scenario.

*Support for recourse investment decisions (e.g., scenario-dependent capacity additions) is planned for a future release.

In [ ]:
# Build stochastic network using PyPSA API
n_stoch = build_network(GAS_PRICES[BASE])

# Add scenarios to the network - this is the key step!
n_stoch.set_scenarios(PROB)

print("\nStochastic network created:")
n_stoch

In [ ]:
print(f"Has scenarios: {n_stoch.has_scenarios}")
print(n_stoch.scenarios)

In [ ]:
# Examine the data structure with scenarios (static data)
n_stoch.generators

In [ ]:
# Examine the data structure with scenarios (time-series data)
n_stoch.generators_t.p_max_pu

In [ ]:
# Set scenario-specific parameters (gas prices vary by scenario)
print("Setting scenario-specific gas prices...")
for sc in SCENARIOS:
    if sc != BASE:
        idx = (sc, "gas")
        # Update gas marginal cost for this scenario
        n_stoch.generators.loc[idx, "marginal_cost"] = (
            GAS_PRICES[sc] / n_stoch.generators.loc[idx, "efficiency"]
        )

print("Gas marginal costs by scenario:")
gas_costs = n_stoch.generators.loc[(slice(None), "gas"), "marginal_cost"]
for sc in SCENARIOS:
    cost = gas_costs.loc[(sc, "gas")]
    print(f"  {sc}: {cost:.1f} EUR/MWh")

In [ ]:
# Solve stochastic optimization problem
print("Solving stochastic optimization problem...")
n_stoch.optimize(solver_name=SOLVER, solver_options={"log_to_console": False})

print(f"Total expected cost: {n_stoch.objective / 1e6:.3f} M€/year")

# Extract results (investment decisions are scenario-independent)
caps_api = n_stoch.generators.p_nom_opt.xs(BASE, level="scenario")
obj_api = n_stoch.objective

In [ ]:
# Inspect the optimization model structure
n_stoch.model

**Variable Structure:**
- `Generator-p_nom`: Investment variables (no scenario dimension) - **First-stage decisions**
- `Generator-p`: Operational variables (scenario, component, snapshot) - **Second-stage decisions**

**Constraint Structure:** Operational constraints are broadcasted across scenarios scenario-specific parameters

In [ ]:
# Create comparison DataFrames
caps_comparison = caps_det.copy()
caps_comparison.loc["Stochastic"] = caps_api

objs_comparison = objs_det.copy()
objs_comparison.loc["Stochastic"] = obj_api

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Capacity mix comparison
colors = [COLOR_MAP.get(c, "gray") for c in caps_comparison.columns]
caps_comparison.plot(kind="bar", stacked=True, ax=ax1, color=colors)
ax1.set_title("Capacity Mix: Deterministic vs. Stochastic")
ax1.set_ylabel("Capacity (MW)")
ax1.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
ax1.tick_params(axis="x", rotation=45)

# Cost comparison
(objs_comparison / 1e6).plot(kind="bar", ax=ax2, color="steelblue")
ax2.set_title("Total Cost: Deterministic vs. Stochastic")
ax2.set_ylabel("Total Cost (M€/year)")
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## Part 3: Manual Stochastic Model Construction

To understand how PyPSA implements stochastic optimization internally, we manually construct the stochastic model using linopy.

In [ ]:
def add_stochastic(n):
    """
    Manually adds scenarios to default PyPSA optimization problem:
    - Declare dispatch variables for additional scenarios
    - Add capacity and nodal balance constraints per scenario
    - Weight objective terms by scenario probabilities
    """
    m = n.optimize.create_model()
    sns = n.snapshots
    c = as_components(n, "Generator")
    cap = m["Generator-p_nom"]  # Investment variables (first-stage)
    min_pu, max_pu = c.get_bounds_pu(attr="p")

    # Add operational variables and constraints for additional scenarios
    for sc in ["med", "high"]:
        # Operational dispatch variables for scenario sc
        var_name = f"Generator-p-{sc}"
        disp = m.add_variables(coords=m["Generator-p"].coords, name=var_name)

        # Generator capacity constraints
        m.add_constraints(disp <= max_pu * cap, name=f"Generator-ext-p-upper-{sc}")
        m.add_constraints(disp >= min_pu * cap, name=f"Generator-ext-p-lower-{sc}")

        # Nodal balance constraints
        buses = n.generators.bus.to_xarray().rename("Bus")
        lhs = (DataArray(1) * disp).groupby(buses).sum()  # Generation by bus
        load = n.components["Load"].da.p_set  # Load by bus
        rhs = load.swap_dims({"name": "Bus"})
        m.add_constraints(lhs == rhs, name=f"Bus-nodal-{sc}")

    # Construct objective function with scenario probabilities
    w = n.snapshot_weightings.objective.loc[sns]  # Time weights
    w_da = DataArray(w.values, coords={"snapshot": sns}, dims=["snapshot"])
    obj_terms = []

    # Operational costs for each scenario
    for sc in SCENARIOS:
        disp = m["Generator-p"] if sc == BASE else m[f"Generator-p-{sc}"]
        cm = n.components["Generator"].da.marginal_cost
        # Update gas price for this scenario
        cm.loc[{"name": "gas"}] = GAS_PRICES[sc] / TECH["gas"]["eff"]
        cm = cm * w_da  # Apply time weights
        obj_terms.append((disp * PROB[sc] * cm).sum())  # Weight by probability

    # Investment costs (scenario-independent)
    cap_cost = c.da.capital_cost
    obj_terms.append((cap * cap_cost).sum())

    # Merge all objective terms
    m.objective = merge(obj_terms)

    return m

In [ ]:
# Build manual stochastic model
print("Building manual stochastic model...")
n_manual_stoch = build_network(GAS_PRICES["low"])
add_stochastic(n_manual_stoch)

In [ ]:
# Solve manual stochastic model
n_manual_stoch.optimize.solve_model(solver_name=SOLVER)

caps_manual_stoch = n_manual_stoch.generators.p_nom_opt
obj_manual_stoch = n_manual_stoch.objective

print(f"Manual stochastic objective: {obj_manual_stoch / 1e6:.3f} M€/year")

## Results Comparison

Let's compare all three approaches and verify that the PyPSA API and manual implementation give identical results.

In [ ]:
# Final comprehensive comparison
caps_final = caps_det.copy()
caps_final.loc["Stochastic API"] = caps_api
caps_final.loc["Stochastic Manual"] = caps_manual_stoch

objs_final = objs_det.copy()
objs_final.loc["Stochastic API"] = obj_api
objs_final.loc["Stochastic Manual"] = obj_manual_stoch

# Verify API and manual implementations match
capacity_diff = (caps_api - caps_manual_stoch).abs().max()
objective_diff = abs(obj_api - obj_manual_stoch)

print("\nVerification:")
print(f"Maximum capacity difference (API vs Manual): {capacity_diff:.6f} MW")
print(f"Objective difference (API vs Manual): {objective_diff:.6f} EUR")
print(f"Implementations match: {capacity_diff < 1e-6 and objective_diff < 1e-3}")

In [ ]:
# Final visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Capacity mix comparison
colors = [COLOR_MAP.get(c, "gray") for c in caps_final.columns]
caps_final.plot(kind="bar", stacked=True, ax=ax1, color=colors)
ax1.set_title("Capacity Mix: All Approaches", fontsize=14, fontweight="bold")
ax1.set_ylabel("Capacity (MW)")
ax1.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
ax1.tick_params(axis="x", rotation=45)
ax1.grid(True, alpha=0.3)

# Cost comparison
(objs_final / 1e6).plot(kind="bar", ax=ax2, color="steelblue", alpha=0.8)
ax2.set_title("Total System Cost: All Approaches", fontsize=14, fontweight="bold")
ax2.set_ylabel("Total Cost (M€/year)")
ax2.tick_params(axis="x", rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Value of Information Analysis

PyPSA supports the evaluation of uncertainty modeling through key metrics from stochastic programming. These metrics quantify the benefits of explicitly modeling uncertainty when planning decisions under unknown future conditions.

---

**Expected Value of Perfect Information (EVPI)**  
Measures the theoretical benefit of having perfect foresight.

$$\text{EVPI} = \text{SP} - \mathbb{E}[\text{WS}]$$

EVPI quantifies the maximum value of knowing uncertain parameters in advance. It is defined as the difference between:

- **SP**: The expected cost from the *stochastic programming* solution, where decisions are made before knowing which scenario will occur.  
- **WS**: The *wait-and-see* solution, where each scenario is solved with perfect information, and the results are averaged.

**Interpretation**:
- **Modeling perspective**: EVPI reflects the cost savings a planner could achieve if future uncertainty were perfectly known.  
- **Economic perspective**: EVPI provides an upper bound on what it would be rational to pay for perfect forecasts.

By definition, EVPI is always non-negative, since perfect information cannot increase expected costs.

---

**Value of the Stochastic Solution (VSS)**  
Also known as the *Expected Cost of Ignoring Uncertainty (ECIU)*.

$$\text{VSS} = \text{EEV} - \text{SP}$$

VSS measures the benefit of accounting for uncertainty, compared to using a deterministic model based on average input values. It is the difference between:

- **EEV**: The expected cost of the *expected value* solution, where decisions are optimized using mean parameter values.  
- **SP**: The cost from the optimal stochastic solution.

**Interpretation**:  
VSS quantifies the penalty of ignoring uncertainty. A higher VSS suggests that stochastic modeling provides substantial improvements over deterministic approaches.

---

## Theoretical Relationships Between VOI Metrics

The Value of Information (VOI) metrics—**Wait-and-See (WS)**, **Stochastic Programming (SP)**, and **Expected Value (EEV)**—are analytically related by well-established results in stochastic programming. For a foundational reference, see Birge (1982) [2].

These core metrics satisfy the following **inequality chain**:

$$
\text{WS} \leq \text{SP} \leq \text{EEV}
$$

- **Wait-and-See (WS)** represents a theoretical lower bound assuming perfect foresight.
- **Stochastic Programming (SP)** models uncertainty directly by optimizing over all scenarios jointly.
- **Expected Value (EEV)** simplifies the problem by using mean input data, potentially at the cost of robustness.

These inequalities imply that:

- **EVPI** and **VSS** are always non-negative.
- Any violation of these relationships typically indicates a modeling or implementation error.

To aid interpretation, EVPI and VSS can also be expressed relative to SP:

- **EVPI / SP**: The potential cost savings from perfect information.
- **VSS / SP**: The benefit of using stochastic over deterministic optimization.

---

Let’s compute these metrics ourselves!

In [ ]:
def calculate_evpi_vss(caps_det, objs_det, obj_stoch, gas_prices, prob):
    """
    Calculate Expected Value of Perfect Information (EVPI) and
    Value of Stochastic Solution (VSS)
    """
    # Expected Value of Perfect Information (EVPI)
    # WS: Wait-and-See (perfect information)
    ws_cost = sum(objs_det[sc] * prob[sc] for sc in SCENARIOS)
    evpi = obj_stoch - ws_cost  # Corrected: SP - E[WS]

    # Value of Stochastic Solution (VSS) / Expected Cost of Ignoring Uncertainty (ECIU)
    # EEV: Expected value of using expected-value solution
    expected_gas_price = sum(gas_prices[sc] * prob[sc] for sc in SCENARIOS)

    # Solve deterministic problem with expected gas price
    n_eev = build_network(expected_gas_price)
    n_eev.optimize(solver_name=SOLVER, solver_options={"log_to_console": False})
    eev_capacities = n_eev.generators.p_nom_opt

    # Evaluate EEV solution under each scenario
    eev_costs = []
    for sc in SCENARIOS:
        n_eval = build_network(gas_prices[sc])
        # Fix capacities to EEV solution
        for tech in TECH.keys():
            n_eval.generators.loc[tech, "p_nom_max"] = eev_capacities[tech]
            n_eval.generators.loc[tech, "p_nom_min"] = eev_capacities[tech]
        n_eval.optimize(solver_name=SOLVER, solver_options={"log_to_console": False})
        eev_costs.append(n_eval.objective)

    eev_expected_cost = sum(eev_costs[i] * prob[sc] for i, sc in enumerate(SCENARIOS))
    vss = eev_expected_cost - obj_stoch  # VSS = EEV - SP

    return {
        "WS": ws_cost,
        "SP": obj_stoch,
        "EEV": eev_expected_cost,
        "EVPI": evpi,
        "VSS": vss,
        "EEV_capacities": eev_capacities,
        "expected_gas_price": expected_gas_price,
    }


# Calculate value of information measures
print("Calculating value of information measures...")
voi_results = calculate_evpi_vss(caps_det, objs_det, obj_api, GAS_PRICES, PROB)

print("\nValue of Information Analysis:")
print("=" * 50)
print(f"Expected gas price: {voi_results['expected_gas_price']:.1f} EUR/MWh_th")
print("\nCost Results (M€/year):")
print(f"  Wait-and-See (WS):     {voi_results['WS'] / 1e6:.3f}")
print(f"  Stochastic Prog (SP):  {voi_results['SP'] / 1e6:.3f}")
print(f"  Expected Value (EEV):  {voi_results['EEV'] / 1e6:.3f}")
print("\nValue Measures:")
print(f"  EVPI (SP - WS):        {voi_results['EVPI'] / 1e6:.3f} M€/year")
print(f"  VSS (EEV - SP):        {voi_results['VSS'] / 1e6:.3f} M€/year")
print("\nRelative Values:")
print(f"  EVPI/SP:               {100 * voi_results['EVPI'] / voi_results['SP']:.2f}%")
print(f"  VSS/SP:                {100 * voi_results['VSS'] / voi_results['SP']:.2f}%")

print("\nTheoretical Ordering Check:")
# https://deepblue.lib.umich.edu/bitstream/handle/2027.42/47912/10107_2005_Article_BF01585113.pdf
print(
    f"  WS ≤ SP: {voi_results['WS'] <= voi_results['SP']} ({voi_results['WS']:.0f} ≤ {voi_results['SP']:.0f})"
)
print(
    f"  SP ≤ EEV: {voi_results['SP'] <= voi_results['EEV']} ({voi_results['SP']:.0f} ≤ {voi_results['EEV']:.0f})"
)
print(f"  EVPI ≥ 0: {voi_results['EVPI'] >= 0} ({voi_results['EVPI']:.0f} ≥ 0)")
print(f"  VSS ≥ 0: {voi_results['VSS'] >= 0} ({voi_results['VSS']:.0f} ≥ 0)")

# Check for violations
if voi_results["EVPI"] < 0:
    print("\nNote: EVPI < 0 violates theory. You're doing something wrong.")

if voi_results["VSS"] < 0:
    print("\nNote: VSS < 0 violates theory. You're doing something wrong.")

In [ ]:
# Visualize value of information
costs_voi = pd.Series(
    {
        "Wait-and-See\n(Perfect Info)": voi_results["WS"],
        "Stochastic\nProgramming": voi_results["SP"],
        "Expected Value\n(Ignore Uncertainty)": voi_results["EEV"],
    }
)

fig, ax1 = plt.subplots(figsize=(7, 5))

# Cost comparison
(costs_voi / 1e6).plot(kind="bar", ax=ax1, color=["green", "blue", "red"], alpha=0.7)
ax1.set_title("Value of Information: Cost Comparison", fontsize=14, fontweight="bold")
ax1.set_ylabel("Total Cost (M€/year)")
ax1.tick_params(axis="x", rotation=45)
ax1.grid(True, alpha=0.3)

# Add EVPI and VSS arrows
ax1.annotate(
    "",
    xy=(0, voi_results["WS"] / 1e6),
    xytext=(1, voi_results["SP"] / 1e6),
    arrowprops={"arrowstyle": "<->", "color": "purple", "lw": 2},
)
ax1.text(
    0.5,
    (voi_results["WS"] + voi_results["SP"]) / (2 * 1e6),
    "EVPI",
    color="purple",
    fontweight="bold",
    fontsize=10,
    ha="center",
)

ax1.annotate(
    "",
    xy=(1, voi_results["SP"] / 1e6),
    xytext=(2, voi_results["EEV"] / 1e6),
    arrowprops={"arrowstyle": "<->", "color": "orange", "lw": 2},
)
ax1.text(
    1.5,
    (voi_results["EEV"] + voi_results["SP"]) / (2 * 1e6),
    "VSS",
    color="orange",
    fontweight="bold",
    fontsize=10,
    ha="center",
)

plt.tight_layout()
plt.show()

## Robust Optimization Connection

Stochastic programming, as implemented in PyPSA, minimizes expected cost across scenarios weighted by their probabilities. This approach provides probabilistic guarantees about system performance and enables risk-neutral decision making. While PyPSA currently focuses on expectation-based optimization, the framework can be extended to incorporate **risk measures such as Conditional Value at Risk (CVaR) or worst-case scenario considerations** to achieve more risk-averse solutions.* 

*Support for risk measures and robust optimization variants is planned in the future release.

## Computational Performance

**Problem Scaling Properties**

- **Investment variables**: Scale as O(|components|)
- **Operational variables**: Scale as O(|scenarios| × |components| × |snapshots|)
- **Constraint matrix structure**: The two-stage stochastic program produces a block-angular matrix. Its size grows linearly with the number of scenarios.

**Memory consumption** increases with the number of scenarios due to:

- Linear growth in the number of variables and constraints
- Solver working memory requirements that may scale super-linearly, depending on the solution algorithm
- Internal solver data structures (e.g. factorization fill-in, basis management) that can exhibit non-linear scaling behavior

### Advanced Decomposition Methods

**PyPSA-SMS++ Integration:**
The [Resilient project](https://resilient-project.github.io/) is developing PyPSA-SMS++, a specialized API that will enable **advanced decomposition algorithms** integrated into PyPSA workflow. The goal is to support large-scale stochastic optimization problems directly within the PyPSA framework.

## References

[1] Birge, J. R., & Louveaux, F. (2011). *Introduction to Stochastic Programming* (2nd ed.). Springer Science & Business Media.

[2] Birge, J. R. (1982). The value of the stochastic solution in stochastic linear programs with fixed recourse. *Mathematical Programming*, 24(1), 314-325. Available at: https://deepblue.lib.umich.edu/bitstream/handle/2027.42/47912/10107_2005_Article_BF01585113.pdf